In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
import torchvision
import torch
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision import transforms
from torch.utils import data
from torch import nn
from torch import optim
from torchvision import models
import torch.nn.functional as F
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
from collections import Counter
import cv2
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
from pyswarms.utils.plotters import plot_cost_history
from PIL import Image, ImageFile
import warnings
warnings.filterwarnings('ignore')

# Advanced Mushroom Classification with EDA, Preprocessing, and PSO Hyperparameter Tuning

This notebook implements:
1. Enhanced Exploratory Data Analysis (EDA)
2. Advanced preprocessing techniques
3. Hyperparameter tuning with Particle Swarm Optimization (PSO)
4. Optimized ResNet50 implementation

In [ ]:
# Handle truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

## Data Loading and Initial Setup

In [ ]:
# Mount Google Drive for Colab if needed
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = "/content/drive/MyDrive/Skripsi-ghamal/Mushrooms"
    COLAB = True
except:
    base_path = "../Mushrooms"  # Adjust this to your local path
    COLAB = False
    
print(f"Running on {'Google Colab' if COLAB else 'Local Machine'}")

In [ ]:
# Set random seeds for reproducibility
def set_seed(seed=1337):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1337)

## Exploratory Data Analysis (EDA)

In [ ]:
# Load the dataset
all_data = datasets.ImageFolder(root=base_path)
class_names = all_data.classes
class_to_idx = all_data.class_to_idx

# Basic dataset info
print(f"Number of mushroom classes: {len(class_names)}")
print(f"Class names: {class_names}")
print(f"Total number of images: {len(all_data)}")

# Distribution of classes
class_counts = Counter([label for _, label in all_data.samples])
class_distribution = {class_names[i]: class_counts[i] for i in range(len(class_names))}

# Plot class distribution
plt.figure(figsize=(14, 6))
sns.barplot(x=list(class_distribution.keys()), y=list(class_distribution.values()))
plt.title('Distribution of Mushroom Classes')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Number of Images')
plt.tight_layout()
plt.show()

In [ ]:
# Function to analyze image properties
def analyze_image_properties(dataset, num_samples=100):
    # Randomly sample images to analyze
    indices = random.sample(range(len(dataset)), min(num_samples, len(dataset)))
    widths, heights, aspect_ratios, sizes, brightness = [], [], [], [], []
    
    for idx in indices:
        img, _ = dataset[idx]
        if not isinstance(img, np.ndarray):
            img_np = np.array(img)
        else:
            img_np = img
            
        # Get dimensions
        if len(img_np.shape) == 3:
            h, w, _ = img_np.shape
        else:
            h, w = img_np.shape
            
        widths.append(w)
        heights.append(h)
        aspect_ratios.append(w/h)
        sizes.append(w*h)
        
        # Calculate average brightness
        if len(img_np.shape) == 3:
            gray_img = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
            brightness.append(np.mean(gray_img))
        else:
            brightness.append(np.mean(img_np))
    
    return {
        'widths': widths,
        'heights': heights,
        'aspect_ratios': aspect_ratios,
        'sizes': sizes,
        'brightness': brightness
    }

# Analyze image properties
img_props = analyze_image_properties(all_data)

# Plot image dimensions distribution
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

axs[0, 0].hist(img_props['widths'], bins=20)
axs[0, 0].set_title('Width Distribution')
axs[0, 0].set_xlabel('Width')

axs[0, 1].hist(img_props['heights'], bins=20)
axs[0, 1].set_title('Height Distribution')
axs[0, 1].set_xlabel('Height')

axs[1, 0].hist(img_props['aspect_ratios'], bins=20)
axs[1, 0].set_title('Aspect Ratio Distribution')
axs[1, 0].set_xlabel('Aspect Ratio (width/height)')

axs[1, 1].hist(img_props['brightness'], bins=20)
axs[1, 1].set_title('Brightness Distribution')
axs[1, 1].set_xlabel('Mean Brightness')

plt.tight_layout()
plt.show()

In [ ]:
# Display sample images from each class
plt.figure(figsize=(15, 12))
for i, class_name in enumerate(class_names):
    # Find first image of this class
    for idx, (img, label) in enumerate(all_data):
        if label == class_to_idx[class_name]:
            plt.subplot(3, 3, i+1)
            plt.imshow(img)
            plt.title(f"{class_name}")
            plt.axis('off')
            break

plt.tight_layout()
plt.show()

## Advanced Image Preprocessing and Data Augmentation

In [ ]:
class MyDataset(data.Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        return self.transform(x), y

    def __len__(self):
        return len(self.subset)

In [ ]:
# Custom center crop with aspect ratio preservation
class CenterCrop(torch.nn.Module):
    def __init__(self, size=None, ratio="1:1"):
        super().__init__()
        self.size = size
        self.ratio = ratio

    def forward(self, img):
        if self.size is None:
            if isinstance(img, torch.Tensor):
                h, w = img.shape[-2:]
            else:
                w, h = img.size
            ratio = self.ratio.split(":")
            ratio = float(ratio[0]) / float(ratio[1])
            ratioed_w = int(h * ratio)
            ratioed_h = int(w / ratio)
            if w>=h:
                if ratioed_h <= h:
                    size = (ratioed_h, w)
                else:
                    size = (h, ratioed_w)
            else:
                if ratioed_w <= w:
                    size = (h, ratioed_w)
                else:
                    size = (ratioed_h, w)
        else:
            size = self.size
        return torchvision.transforms.functional.center_crop(img, size)

In [ ]:
# Define advanced augmentation for training data
def get_transforms(img_size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    # Training transforms with strong augmentation
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(img_size, scale=(0.25, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    # Test transforms - just resize and normalize
    test_transforms = transforms.Compose([
        CenterCrop(),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    return train_transforms, test_transforms

In [ ]:
# Split data with stratification to maintain class distribution - Optimized version
from sklearn.model_selection import train_test_split
import time

def stratified_split_fast(dataset, test_size=0.3, random_state=42):
    """
    Faster implementation of stratified split that avoids loading all images
    by using the class information already stored in the dataset.samples
    """
    start_time = time.time()
    print("Starting stratified split...")
    
    # Extract labels directly from dataset.samples which doesn't load actual images
    indices = list(range(len(dataset)))
    
    # This is much faster since dataset.samples already contains labels
    # without loading the actual images
    labels = [dataset.samples[i][1] for i in indices]
    
    print(f"Extracted labels in {time.time() - start_time:.2f} seconds")
    
    # Perform the split
    split_start = time.time()
    train_idx, test_idx = train_test_split(
        indices, 
        test_size=test_size, 
        random_state=random_state,
        stratify=labels
    )
    print(f"Split performed in {time.time() - split_start:.2f} seconds")
    
    # Create subset without accessing actual images yet
    print(f"Total time: {time.time() - start_time:.2f} seconds")
    return torch.utils.data.Subset(dataset, train_idx), torch.utils.data.Subset(dataset, test_idx)

# Split the data using the optimized function
print("Using optimized stratified split function")
train_data, test_data = stratified_split_fast(all_data, test_size=0.3)

print(f"Training set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

# Get transforms
train_transforms, test_transforms = get_transforms()

# Apply transforms
print("Applying transforms to datasets...")
train_data = MyDataset(train_data, train_transforms)
test_data = MyDataset(test_data, test_transforms)
print("Done!")

In [ ]:
# Visualize augmentations
def show_augmentations(image, transform, num_samples=5):
    """Apply the transformation multiple times to show its effect"""
    fig, axes = plt.subplots(1, num_samples + 1, figsize=(16, 4))
    
    # Display original image
    axes[0].imshow(image)
    axes[0].set_title("Original")
    axes[0].axis('off')
    
    # Display transformations
    for i in range(num_samples):
        transformed_img = transform(image)
        if isinstance(transformed_img, torch.Tensor):
            # Convert tensor to numpy for displaying
            img_np = transformed_img.numpy().transpose((1, 2, 0))
            # Un-normalize the image
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img_np = std * img_np + mean
            img_np = np.clip(img_np, 0, 1)
        else:
            img_np = np.array(transformed_img)
        
        axes[i+1].imshow(img_np)
        axes[i+1].set_title(f"Aug {i+1}")
        axes[i+1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Get a sample image
sample_img = all_data[0][0]
show_augmentations(sample_img, train_transforms, num_samples=5)

## GPU Setup

In [ ]:
# Check GPU availability
torch.cuda.is_available()

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    device = torch.device("cpu")
    print("No GPU available. Training will run on CPU.")

## Hyperparameter Optimization with Particle Swarm Optimization (PSO)

In [ ]:
# Define ResNet50 model architecture with configurable hyperparameters
def create_model(lr=0.01, momentum=0.9, weight_decay=0.0001, unfreeze_layers=1):
    """Create a ResNet50 model with specified hyperparameters"""
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    
    # Freeze all layers by default
    for param in model.parameters():
        param.requires_grad = False
    
    # Unfreeze specified number of layers from the end
    layers_to_unfreeze = []
    if unfreeze_layers >= 1:
        layers_to_unfreeze.append(model.layer4)
    if unfreeze_layers >= 2:
        layers_to_unfreeze.append(model.layer3)
    if unfreeze_layers >= 3:
        layers_to_unfreeze.append(model.layer2)
    if unfreeze_layers >= 4:
        layers_to_unfreeze.append(model.layer1)
    
    for layer in layers_to_unfreeze:
        for param in layer.parameters():
            param.requires_grad = True
    
    # Replace the final fully connected layer
    num_classes = len(all_data.classes)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    # Move model to device
    model = model.to(device)
    
    # Define optimizer
    optimizer = optim.SGD(
        model.parameters(),
        lr=lr,
        momentum=momentum,
        weight_decay=weight_decay
    )
    
    # Define criterion (loss function)
    criterion = nn.CrossEntropyLoss().to(device)
    
    return model, optimizer, criterion

In [ ]:
# Create data loaders for validation
def create_data_loaders(batch_size=64, val_size=0.1):
    """Create train, validation, and test data loaders"""
    # Further split training data to create a validation set
    train_indices = list(range(len(train_data)))
    val_indices = train_indices[:int(val_size * len(train_indices))]
    train_indices = train_indices[int(val_size * len(train_indices)):]
    
    train_subset = torch.utils.data.Subset(train_data, train_indices)
    val_subset = torch.utils.data.Subset(train_data, val_indices)
    
    # Create data loaders
    train_loader = data.DataLoader(
        train_subset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=2 if COLAB else 0
    )
    
    val_loader = data.DataLoader(
        val_subset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2 if COLAB else 0
    )
    
    test_loader = data.DataLoader(
        test_data, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=2 if COLAB else 0
    )
    
    return train_loader, val_loader, test_loader

In [ ]:
# Quick model training and validation functions for PSO
def train_quick(model, optimizer, criterion, train_loader, max_batches=50):
    """Quick training function for hyperparameter optimization"""
    model.train()
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader):
        if i >= max_batches:
            break
            
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    return running_loss / min(max_batches, len(train_loader))

def validate_quick(model, val_loader, max_batches=30):
    """Quick validation function for hyperparameter optimization"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            if i >= max_batches:
                break
                
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return correct / total

In [ ]:
# Create data loaders for PSO
train_loader, val_loader, test_loader = create_data_loaders(batch_size=32)

# Define PSO objective function
def pso_objective_function(params):
    """Objective function to minimize for PSO"""
    n_particles = params.shape[0]
    performance = []
    
    for i in range(n_particles):
        # Extract hyperparameters
        lr = 10 ** params[i, 0]  # log scale for learning rate: 10^-4 to 10^-1
        momentum = params[i, 1]  # 0.8 to 0.99
        weight_decay = 10 ** params[i, 2]  # log scale: 10^-5 to 10^-3
        unfreeze_layers = int(params[i, 3]) + 1  # 1 to 4 layers
        
        # Create model with these hyperparameters
        model, optimizer, criterion = create_model(
            lr=lr, 
            momentum=momentum,
            weight_decay=weight_decay,
            unfreeze_layers=unfreeze_layers
        )
        
        # Train for a few iterations
        train_loss = train_quick(model, optimizer, criterion, train_loader)
        
        # Validate
        val_acc = validate_quick(model, val_loader)
        
        # We want to minimize (negative of validation accuracy + small weight for training loss)
        performance.append(-val_acc + 0.1 * train_loss)
        
    return np.array(performance)

In [ ]:
# Run PSO for hyperparameter optimization
# This is a resource-intensive step, so we limit particles and iterations

# Setup hyperparameter bounds
# [log_lr, momentum, log_weight_decay, unfreeze_layers]
min_bounds = np.array([-4.0, 0.8, -5.0, 0])  # Lower bounds
max_bounds = np.array([-1.0, 0.99, -3.0, 3])  # Upper bounds
bounds = (min_bounds, max_bounds)

# PSO options
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 2, 'p': 2}

# Initialize swarm with fewer particles for demo
n_particles = 5
dimensions = 4
optimizer = ps.single.GlobalBestPSO(
    n_particles=n_particles,
    dimensions=dimensions,
    options=options,
    bounds=bounds
)

# Run optimization (limited iterations for demo)
print("Starting PSO optimization. This may take some time...")
cost, pos = optimizer.optimize(pso_objective_function, iters=3)

# Convert best position to actual hyperparameters
best_lr = 10 ** pos[0]
best_momentum = pos[1]
best_weight_decay = 10 ** pos[2]
best_unfreeze_layers = int(pos[3]) + 1

print(f"Best hyperparameters found:")
print(f"Learning Rate: {best_lr:.6f}")
print(f"Momentum: {best_momentum:.4f}")
print(f"Weight Decay: {best_weight_decay:.6f}")
print(f"Layers to Unfreeze: {best_unfreeze_layers}")

## Model Training with Optimal Hyperparameters

In [ ]:
# Create final model with best hyperparameters
model, optimizer, criterion = create_model(
    lr=best_lr,
    momentum=best_momentum,
    weight_decay=best_weight_decay,
    unfreeze_layers=best_unfreeze_layers
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Full training function
def train_model(model, optimizer, criterion, scheduler, train_loader, val_loader, epochs=10):
    """Full training function with validation"""
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    best_val_acc = 0
    best_model_wts = None
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for inputs, labels in train_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            train_bar.set_postfix(loss=loss.item())
            
        epoch_train_loss = running_loss / len(train_loader)
        train_losses.append(epoch_train_loss)
        
        # Validation phase
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Valid]")
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                val_bar.set_postfix(loss=loss.item())
        
        epoch_val_loss = running_loss / len(val_loader)
        val_losses.append(epoch_val_loss)
        
        epoch_val_acc = correct / total
        val_accuracies.append(epoch_val_acc)
        
        print(f"Epoch {epoch+1}/{epochs}:")
        print(f"  Train Loss: {epoch_train_loss:.4f}")
        print(f"  Val Loss: {epoch_val_loss:.4f}")
        print(f"  Val Accuracy: {epoch_val_acc:.4f}")
        
        # Save best model
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            best_model_wts = model.state_dict().copy()
        
        # Step the scheduler
        scheduler.step()
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, train_losses, val_losses, val_accuracies

In [ ]:
# Train the model
epochs = 10
model, train_losses, val_losses, val_accuracies = train_model(
    model, optimizer, criterion, scheduler, train_loader, val_loader, epochs=epochs
)

# Save the model
torch.save(model.state_dict(), "mushroom_optimized_model.pt")

# Plot training and validation curves
plt.figure(figsize=(12, 4))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs+1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## Model Evaluation

In [ ]:
# Evaluate the model on the test set
def evaluate_model(model, test_loader):
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    return all_preds, all_labels

# Get predictions
predictions, truth = evaluate_model(model, test_loader)

# Calculate accuracy
accuracy = accuracy_score(truth, predictions)
print(f"Test Accuracy: {accuracy:.4f}")

# Create and display confusion matrix
cm = confusion_matrix(truth, predictions)
plt.figure(figsize=(12, 10))
ax = sns.heatmap(
    cm / np.sum(cm, axis=1)[:, np.newaxis], 
    annot=True, 
    fmt='.2%', 
    cmap='Blues',
    xticklabels=class_names, 
    yticklabels=class_names
)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# Detailed classification report
print("\nClassification Report:")
print(classification_report(truth, predictions, target_names=class_names))

In [ ]:
# Visualize some of the model's predictions
def plot_predictions(model, test_loader, class_names, num_samples=10):
    """Plot some predictions from the model"""
    model.eval()
    
    all_images = []
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            # Collect samples
            for i in range(inputs.size(0)):
                all_images.append(inputs[i].cpu())
                all_labels.append(labels[i].item())
                all_preds.append(preds[i].cpu().item())
                
                if len(all_images) >= num_samples:
                    break
            
            if len(all_images) >= num_samples:
                break
    
    # Display predictions
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.flatten()
    
    for i, (img, true_label, pred_label) in enumerate(zip(all_images, all_labels, all_preds)):
        if i >= num_samples:
            break
            
        img_np = img.numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img_np = std * img_np + mean
        img_np = np.clip(img_np, 0, 1)
        
        ax = axes[i]
        ax.imshow(img_np)
        
        title_color = 'green' if true_label == pred_label else 'red'
        ax.set_title(f"True: {class_names[true_label]}\nPred: {class_names[pred_label]}", 
                     color=title_color)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize predictions
plot_predictions(model, test_loader, class_names, num_samples=10)

## Feature Visualization and Interpretation

In [ ]:
# Function to extract features from the penultimate layer
def extract_features(model, data_loader):
    """Extract features from the penultimate layer"""
    # Create a new model that outputs features from the penultimate layer
    feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
    feature_extractor.eval()
    
    features = []
    labels = []
    
    with torch.no_grad():
        for inputs, batch_labels in tqdm(data_loader, desc="Extracting features"):
            inputs = inputs.to(device)
            batch_features = feature_extractor(inputs)
            batch_features = batch_features.view(batch_features.size(0), -1)
            
            features.append(batch_features.cpu().numpy())
            labels.append(batch_labels.numpy())
    
    features = np.vstack(features)
    labels = np.concatenate(labels)
    
    return features, labels

# Get features from test set
test_features, test_labels = extract_features(model, test_loader)

# Reduce dimensionality for visualization
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# PCA reduction
pca = PCA(n_components=2)
pca_result = pca.fit_transform(test_features)

# t-SNE reduction
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(test_features)

# Plot PCA results
plt.figure(figsize=(12, 10))
plt.subplot(2, 1, 1)
for i, class_name in enumerate(class_names):
    indices = test_labels == i
    plt.scatter(pca_result[indices, 0], pca_result[indices, 1], label=class_name)
plt.title('PCA Feature Visualization')
plt.legend()

# Plot t-SNE results
plt.subplot(2, 1, 2)
for i, class_name in enumerate(class_names):
    indices = test_labels == i
    plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1], label=class_name)
plt.title('t-SNE Feature Visualization')
plt.legend()

plt.tight_layout()
plt.show()

## Model Deployment Tools

In [ ]:
# Create a simple inference function for deployment
def predict_mushroom_type(model, image_path, class_names):
    """Predict mushroom type from an image file"""
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    
    # Apply the same transformations as for the test set
    transform = transforms.Compose([
        CenterCrop(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Make prediction
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = F.softmax(output, dim=1)
        
        # Get top probabilities and classes
        top_probs, top_classes = torch.topk(probabilities, 3)
    
    # Prepare results
    results = []
    for i in range(3):
        class_idx = top_classes[0][i].item()
        prob = top_probs[0][i].item()
        results.append({
            'class': class_names[class_idx],
            'probability': f"{prob:.4f}"
        })
    
    return results

# Demo with a sample image from the test set
sample_idx = random.randint(0, len(test_data) - 1)
sample_img, sample_label = test_data[sample_idx]

# Save sample image to a temporary file
import tempfile
temp_img = tempfile.NamedTemporaryFile(suffix='.jpg')
temp_img_path = temp_img.name

# Convert tensor to PIL image and save
sample_img_np = sample_img.numpy().transpose((1, 2, 0))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
sample_img_np = std * sample_img_np + mean
sample_img_np = np.clip(sample_img_np, 0, 1)
sample_img_pil = Image.fromarray((sample_img_np * 255).astype(np.uint8))
sample_img_pil.save(temp_img_path)

# Make prediction
predictions = predict_mushroom_type(model, temp_img_path, class_names)

# Display results
plt.figure(figsize=(6, 6))
plt.imshow(sample_img_pil)
plt.title(f"True class: {class_names[sample_label]}")
plt.axis('off')

print("Top 3 predictions:")
for i, pred in enumerate(predictions):
    print(f"{i+1}. {pred['class']} (Probability: {pred['probability']})")

## Conclusion

In this notebook, we've implemented:

1. **Advanced EDA** - Analysis of class distribution and image properties
2. **Enhanced Image Preprocessing** - Robust augmentation and preprocessing techniques 
3. **PSO Hyperparameter Optimization** - Finding optimal hyperparameters for the ResNet50 model
4. **Model Training and Evaluation** - Training with the best hyperparameters and detailed evaluation
5. **Feature Visualization** - Understanding what the model has learned through feature space visualization
6. **Deployment-Ready Code** - Simple inference function for making predictions on new images

This comprehensive approach resulted in an optimized mushroom classification model with improved accuracy and robustness.